In [0]:
import numpy as np
import pandas as pd
import datetime
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

start_time = datetime.datetime.now()

# 1. Загрузим данные: в X_train запишем признаки, не связанные с итогами матча, в y_train - исходы матчей
data = pd.read_csv('./features.csv', index_col='match_id')
X_train = pd.DataFrame(data.loc[:, 'start_time':'dire_first_ward_time'])
y_train = pd.DataFrame(data['radiant_win'])
X_test = pd.read_csv('./features_test.csv', index_col='match_id')

# 1.1 Заполним NaN значения нулями
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# 1.2 Выполним масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 1.3 Выберем разбиение
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# 1.4 Обучим классификатор и проверим качество на кросс-валидации
for coef in [0.01, 0.1, 0.5, 1.0, 3.0, 10.0]:
  clf = LogisticRegression(penalty='l2', C=coef)
  clf.fit(X_train_scaled, y_train)
  c_v = cross_val_score(clf, X_train_scaled, y_train, cv=kf, scoring='roc_auc')
  print('{c} L2 coef = {p}'.format(c = round(c_v.mean(), 6), p=coef))
  print ('Time elapsed:', datetime.datetime.now() - start_time)

#0.71655  L2 coef = 0.01 Время выполнения ~ 35s
#0.716527 L2 coef = 0.1
#0.716523 L2 coef = 0.5 
#0.716523 L2 coef = 1.0 
#0.716522 L2 coef = 3.0
#0.716522 L2 coef = 10.0

# 2.1 Удалим категориальный признаки

X_train_ = X_train.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', \
                                                      'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])
X_test_ = X_test.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', \
                                                      'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])

scaler_ = StandardScaler()
X_train_scaled_ = scaler.fit_transform(X_train_)
X_test_scaled_ = scaler.transform(X_test_)

# 2.2 Обучим классификатор и проверим качество на кросс-валидации
for coef in [0.01, 0.1, 0.5, 1.0, 3.0, 10.0]:
  clf = LogisticRegression(penalty='l2', C=0.01)
  clf.fit(X_train_scaled_, y_train)
  c_v = cross_val_score(clf, X_train_scaled_, y_train, cv=kf, scoring='roc_auc')
  print('{c} L2 coef = {p}'.format(c = round(c_v.mean(), 6), p=coef))
  print ('Time elapsed:', datetime.datetime.now() - start_time)
  
#0.716559 L2 coef = 0.01 Время выполнения ~ 30s
#0.716534 L2 coef = 0.1
#0.716531 L2 coef = 0.5
#0.71653  L2 coef = 1.0
#0.71653  L2 coef = 3.0
#0.71653  L2 coef = 10.0

# В сравнении с градиентным бустингом получается результат с меньшим значением качества на кросс-валидации
# Объяснение - логистическая регрессия - линейный классификатор, а градиентный бустинг - нелинейный.
# Градиентный бустинг лучше восстанавливает нелинейные зависимости

# Время работы логистической регрессии существенно ниже времени работы градиентного бустинга при большом количестве деревьев

#Видим, что качество улучшилось, но незначительно, в 3-4 знаке, вывод - исключение категориальных признаков не вносит существенный вклад
#Как можно объяснить - логистическая регрессия не дает существенные веса признакам, которые не вносят существенный вклад в предсказание

# 3. N — количество различных героев в выборке
# Взял max, чтобы не падало на цикле ниже, так как есть пропуски по героям в исходной выборке
# Согласно справочника героев, всего их 113, в нашем случае 112 герой - максимальный
N = X_train['r1_hero'].unique().max()

# 4.1 Воспользуемся подходом 'мешок слов' для train и test (test нужен для получения финального предсказания)
X_pick_train = np.zeros((X_train.shape[0], N))
X_pick_test = np.zeros((X_test.shape[0], N))

for i, match_id in enumerate(X_train.index):
    for p in range(5):
        X_pick_train[i, X_train.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_train[i, X_train.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick_test[i, X_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, X_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X_pick_train = pd.DataFrame(X_pick_train, index = X_train.index)
X_pick_test = pd.DataFrame(X_pick_test, index = X_test.index)

# 4.2 Добавим наши новые признаки, очистим от неинформативных признаков, выполним масштабирование
X_train_add = pd.concat((X_train, X_pick_train), axis = 1)
X_test_add = pd.concat((X_test, X_pick_test), axis = 1)

X_train_add_ = X_train_add.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', \
                                                      'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])
X_test_add_ = X_test_add.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', \
                                                      'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])

scaler_ = StandardScaler()
X_train_scaled_add = scaler.fit_transform(X_train_add_)
X_test_scaled_add = scaler.transform(X_test_add_)

# 5.1 Обучим классификатор и проверим качество на кросс-валидации
#for coef in [0.01, 0.1, 0.5, 1.0, 3.0, 10.0]:
clf = LogisticRegression(penalty='l2', C=0.01)
clf.fit(X_train_scaled_add, y_train)
c_v = cross_val_score(clf, X_train_scaled_add, y_train, cv=kf, scoring='roc_auc')
print('{c} L2 coef = {p}'.format(c = round(c_v.mean(), 6), p=coef))
print ('Time elapsed:', datetime.datetime.now() - start_time)

# 0.751964 L2 coef = 0.01
# Видим, что качество значительно улучшилось, то есть наше предположение, о том, что некоторые выигрывают чаще, оказалось верным

# 6. Построим предсказания:
pred = pd.Series(clf.predict_proba(X_test_scaled_add)[:, 1], name = 'radiant_win', index = X_test.index)

# >> c_v
# array([0.74934397, 0.75315468, 0.7495552 , 0.75609852, 0.75166785])
# модель адекватная, получилась не константной

# 7. Сохраним наши результаты и загрузим на kaggle
#X_test_final = pd.concat([X_test, pred], axis=1, ignore_index = True)
#res = pd.Series(X_test_final[102])
#res.to_csv('./submission_log.csv')
#грузим на kaggle, получаем score = 0.75526 на public leaderbord